In [2]:
from FinMind.data import DataLoad

api = DataLoader()
api.login_by_token(api_token='token')

ImportError: cannot import name 'DataLoad' from 'FinMind.data' (/opt/anaconda3/lib/python3.11/site-packages/FinMind/data/__init__.py)